# day 20: Particle Swarm

Answer to part one is simply: of particles with smallest accels, choose particles with smallest velocities, and then with closest positions (as needed in case of ties).

In [1]:
test_input = '''
p=< 3,0,0>, v=< 2,0,0>, a=<-1,0,0>
p=< 4,0,0>, v=< 0,0,0>, a=<-2,0,0>
'''

# particle 0 has smaller abs accel than 1, so it stays closest over time

In [2]:
particle_specs = []
with open('input_day_20.txt') as f:
    for line in f:
        particle_specs.append(line.strip('\n'))

In [3]:
def get_abs_accel(particle_spec):
    aa = sum([abs(int(ac)) for ac in (particle_spec.split()[2][3:-1]).split(',')])
    return aa
    
accel_part = sorted([(get_abs_accel(p), pi) for pi, p in enumerate(particle_specs)])
min_accel_part_inds = [ap[1] for ap in accel_part if ap[0] == accel_part[0][0]]
ma_particles = [(ind, particle_specs[ind]) for ind in min_accel_part_inds]
ma_particles

[(279, 'p=<-1103,92,1785>, v=<49,-4,-97>, a=<1,0,0>'),
 (308, 'p=<2978,2082,4280>, v=<-135,-88,-178>, a=<1,0,0>'),
 (435, 'p=<2030,-4343,-355>, v=<-69,145,25>, a=<0,0,-1>')]

By inspection, particle 308 has larger initial velocity than 279 or 435, so of the three particles with minimum acceleration, it will stay closest.


Part two is not so trivial, here we need to do the actual simulation, and we can stop when the distance ordering by (accel, vel, dist) is equal to the ordering by (dist).

In [4]:
def spec_to_particle(particle_spec):
    coll = False
    p, v, a = particle_spec.split(', ')
    p = [int(n) for n in p[3:-1].split(',')]
    v = [int(n) for n in v[3:-1].split(',')]
    a = [int(n) for n in a[3:-1].split(',')]
    return coll, p, v, a


def effect_collision(particle, col_locs):
    coll, p, v, a = particle
    if not coll:
        if p in col_locs:
            coll = True
            v = [0, 0, 0]
            a = [0, 0, 0]
    return coll, p, v, a


def check_for_collisions(particle_list):
    collision_locations = []
    seen_locations = []
    for particle in particle_list:
        coll, p, v, a = particle
        if not coll:
            if p in seen_locations:
                collision_locations.append(p)
            else:
                seen_locations.append(p)
    return collision_locations


def timestep_particle(particle):
    coll, p, v, a = particle
    if not coll:
        v[0] += a[0]
        p[0] += v[0]
        v[1] += a[1]
        p[1] += v[1]
        v[2] += a[2]
        p[2] += v[2]
    return coll, p, v, a


particle_list = [spec_to_particle(ps) for ps in particle_specs]

# try this before implementing the stricter ordering convergence test
cts = []
for ts in range(100):
    col_locs = check_for_collisions(particle_list)
    if col_locs:
        cts.append(ts)
        particle_list = [effect_collision(p, col_locs) for p in particle_list]
    particle_list = [timestep_particle(p) for p in particle_list] 

print(cts)
print(sum([not p[0] for p in particle_list]))

[10, 11, 13, 14, 15, 16, 17, 18, 19, 20, 21, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39]
504


# day 19: A Series of Tubes

In [1]:
from collections import defaultdict


def map_from_rows(rows):
    map_dict = defaultdict(str)
    for r, row in enumerate(rows):
        for c, col in enumerate(row):
            map_dict[r, c] = col
    return map_dict


def find_start_col(row_0):
    for c, cc in enumerate(row_0):
        if cc == '|':
            return c
        

def traverse_from_rows(rows):
    up, down, left, right = ((-1, 0), (1, 0), (0, -1), (0, 1))
    letters_seen = []

    map_dict = map_from_rows(rows)
    r = 0
    c = find_start_col(rows[0])
    ch = map_dict[r, c]
    direction = down
    ended = False
    steps = 0

    while not ended and ch not in (' ', ''):
        steps += 1
        if ch.isalpha():
            letters_seen.append(ch)
        elif ch == '+':
            if direction in (down, up):
                left_ch = map_dict[r, c-1]
                right_ch = map_dict[r, c+1]
                if left_ch == '-' or (left_ch.isalpha() and right_ch != '-'):
                    direction = left
                elif right_ch == '-' or right_ch.isalpha():
                    direction = right
                else:
                    ended = True
            else:  # direction in (left, right):
                up_ch = map_dict[r-1, c]
                down_ch = map_dict[r+1, c]
                if up_ch == '|' or (up_ch.isalpha() and down_ch != '|'):
                    direction = up
                elif down_ch == '|' or down_ch.isalpha():
                    direction = down
                else:
                    ended = True
        r, c = r + direction[0], c + direction[1]
        ch = map_dict[r, c]

    return ''.join(letters_seen), steps

In [2]:
test_input = """
     |          
     |  +--+    
     A  |  C    
 F---|----E|--+ 
     |  |  |  D 
     +B-+  +--+ 
"""

rows = test_input.split('\n')[1:-1]

traverse_from_rows(rows)

('ABCDEF', 38)

In [3]:
rows = []
with open('input_day_19.txt') as f:
    for line in f:
        rows.append(line.strip('\n'))

traverse_from_rows(rows)

('SXWAIBUZY', 16676)

# day 18: Duet

In [1]:
from collections import defaultdict


def do_instruction(instr):
    offset = 1
    
    ins, *args = instr.split()
    if len(args) == 1:
        [reg] = args
    else:
        reg, arg = args
        if arg.isalpha():
            arg = registers[arg]
        arg = int(arg)
        
    if ins == 'snd':
        registers['played'] = registers[reg]
    elif ins == 'set':
        registers[reg] = arg
    elif ins == 'add':
        registers[reg] += arg
    elif ins == 'mul':
        registers[reg] = registers[reg] * arg
    elif ins == 'mod':
        registers[reg] = registers[reg] % arg
    elif ins == 'rcv':
        freq = registers[reg]
        if freq:
            registers['recovered'] = freq
            print(f"last played freq = {registers['played']}")
            offset += 2*len(instructions)
    elif ins == 'jgz':
        if reg.isalpha():
            val = registers[reg]
        else:
            val = int(reg)
        if val > 0:
            offset = int(arg)
            
    return offset

In [2]:
test_input = '''set a 1
add a 2
mul a a
mod a 5
snd a
set a 0
rcv a
jgz a -1
set a 1
jgz a -2'''
instructions = test_input.split('\n')

In [3]:
registers = defaultdict(int)
cur_ins = 0
while 0 <= cur_ins <= len(instructions):
    cur_ins += do_instruction(instructions[cur_ins])
    
registers

last played freq = 4


defaultdict(int, {'a': 1, 'played': 4, 'recovered': 1})

In [4]:
instructions = []
with open('input_day_18.txt') as f:
    for line in f:
        instructions.append(line.strip())

In [5]:
registers = defaultdict(int)
cur_ins = 0
while 0 <= cur_ins <= len(instructions):
    cur_ins += do_instruction(instructions[cur_ins])
    
registers

last played freq = 3423


defaultdict(int,
            {'a': 2147483647,
             'b': 3423,
             'f': 0,
             'i': 126,
             'p': 1254123423,
             'played': 3423,
             'recovered': 2147483647})

## part two

The Executor class isn't terminating on simultaneous 'rcv' like it should, but the right answer is obtained on interrupt ...



In [6]:
class Executor:
    def __init__(self):
        self.registers = defaultdict(int)
        self.messages = []
        self.instructions = None
        self.send_ct = 0
        self.cur_ins = 0
        self.corresponder = None
        self.waiting = False
        self.terminated = False
        
    def nudge(self):
        if self.messages:
            self.waiting = False
        while not self.terminated and not self.waiting:
            try: 
                self.do_instruction(self.instructions[self.cur_ins])
            except IndexError:
                self.terminated = True
    
    def do_instruction(self, instr):
        offset = 1
        ins, *args = instr.split()
        if len(args) == 1:
            [reg] = args
        else:
            reg, arg = args
            if arg.isalpha():
                arg = self.registers[arg]
            arg = int(arg)
        if ins == 'snd':
            if reg.isalpha():
                reg = self.registers[reg]
            self.corresponder.messages.append(int(reg))
            self.send_ct += 1
        elif ins == 'set':
            self.registers[reg] = arg
        elif ins == 'add':
            self.registers[reg] += arg
        elif ins == 'mul':
            self.registers[reg] = self.registers[reg] * arg
        elif ins == 'mod':
            self.registers[reg] = self.registers[reg] % arg
        elif ins == 'rcv':
            if self.messages:
                val = self.messages.pop(0)
                self.registers[reg] = val
            else:
                offset = 0
                self.waiting = True
        elif ins == 'jgz':
            if reg.isalpha():
                val = self.registers[reg]
            else:
                val = int(reg)
            if val > 0:
                offset = int(arg)
        self.cur_ins += offset

In [7]:
exec_one = Executor()
exec_two = Executor()

exec_one.corresponder = exec_two
exec_two.corresponder = exec_one

exec_one.registers['p'] = 0
exec_one.registers['p'] = 1

exec_one.instructions = instructions
exec_two.instructions = instructions

while not (exec_one.terminated and exec_two.terminated):
    exec_one.nudge()
    exec_two.nudge()

KeyboardInterrupt: 

In [8]:
exec_one.send_ct

7493

# day 17: Spinlock

In [1]:
fwd_steps = 3

buffer = [0]
cur_pos = 0
for ins in range(1, 10):
    cur_pos = (cur_pos + fwd_steps) % len(buffer)
    cur_pos += 1
    if cur_pos > len(buffer)-1:
        buffer.append(ins)
    else:
        buffer = buffer[:cur_pos] + [ins] + buffer[cur_pos:]
    print(buffer)

[0, 1]
[0, 2, 1]
[0, 2, 3, 1]
[0, 2, 4, 3, 1]
[0, 5, 2, 4, 3, 1]
[0, 5, 2, 4, 3, 6, 1]
[0, 5, 7, 2, 4, 3, 6, 1]
[0, 5, 7, 2, 4, 3, 8, 6, 1]
[0, 9, 5, 7, 2, 4, 3, 8, 6, 1]


In [2]:
%%time
fwd_steps = 344

buffer = [0]
cur_pos = 0
for ins in range(1, 2018):
    cur_pos = (cur_pos + fwd_steps) % len(buffer)
    cur_pos += 1
    if cur_pos > len(buffer)-1:
        buffer.append(ins)
    else:
        buffer = buffer[:cur_pos] + [ins] + buffer[cur_pos:]

CPU times: user 23 ms, sys: 1.08 ms, total: 24.1 ms
Wall time: 23.2 ms


In [3]:
buffer[cur_pos-2:cur_pos+3]

[337, 1071, 2017, 996, 1155]

Another *trick* question, where we don't have the time/efficiency (or more importantly, need) to simply extend the part one solution to the larger number of itertions ...

As we only need the value after 0, and as it appears nothing is ever inserted in position 0, we simply need keep track of the value in position 1.

In [4]:
%%time
fwd_steps = 344

pos_zero = 0
buffer_len = 1
pos_one = 0
cur_pos = 0
for ins in range(1, 50_000_000+1):
    cur_pos = (cur_pos + fwd_steps) % buffer_len
    cur_pos += 1
    if cur_pos == 0:
        raise IndexError
    elif cur_pos == 1:
        pos_one = ins
    buffer_len += 1
print(pos_one)

1898341
CPU times: user 21.6 s, sys: 128 ms, total: 21.8 s
Wall time: 22.3 s


# day 16: Permutation Promenade

In [1]:
def do_op(op):
    global prog_list
    op_kind = op[0]
    op_targ = op[1:]
    if op_kind == 's':
        spin_len = int(op_targ)
        prog_list = prog_list[-spin_len:] + prog_list[:-spin_len]
    elif op_kind == 'x':
        sp1, sp2 = [int(n) for n in op_targ.split('/')]
        tmp = prog_list[sp1]
        prog_list[sp1] = prog_list[sp2]
        prog_list[sp2] = tmp
    elif op_kind == 'p':
        pp1, pp2 = op_targ.split('/')
        sp1 = prog_list.index(pp1)
        sp2 = prog_list.index(pp2)
        tmp = prog_list[sp1]
        prog_list[sp1] = prog_list[sp2]
        prog_list[sp2] = tmp

In [2]:
prog_list = list('abcde')
print(prog_list)
do_op('s1')
print(prog_list)
do_op('x3/4')
print(prog_list)
do_op('pe/b')
print(prog_list)

['a', 'b', 'c', 'd', 'e']
['e', 'a', 'b', 'c', 'd']
['e', 'a', 'b', 'd', 'c']
['b', 'a', 'e', 'd', 'c']


In [3]:
with open('input_day_16.txt') as f:
    dance_steps = f.read().strip().split(',')
len(dance_steps)

10000

In [4]:
alpha_16 = 'abcdefghijklmnop'
len(alpha_16)

16

In [5]:
%%time
prog_list = list(alpha_16)
for step in dance_steps:
    do_op(step)
post_dance = ''.join(prog_list)
print(f'Answer to Part One is: {post_dance}')

Answer to Part One is: kpbodeajhlicngmf
CPU times: user 14.3 ms, sys: 295 µs, total: 14.6 ms
Wall time: 14.4 ms


## 16p2

We can't naively execute the 10k step dance 1 billion times, it would take far, far too long.

Also, we can't simply repeat the 10k position remapping, because there are value-based as well as position-based changes.

Q: Maybe there are cycles?
- first, rewrite for a bit of a speedup by:
    - pre-processing the dance move list
    - consolidating the code into a single list, to avoid the (negligible?) function-call overhead
    
A: The cycle exists, and it is surprisingly short

In [6]:
len(set(dance_steps))

495

In [7]:
step_func_dict = {}
for step in set(dance_steps):
    move = step[0]
    args = step[1:].split('/')
    if move != 'p':
        args = [int(n) for n in args]
    step_func_dict[step] = (move, tuple(args))

x_dance_steps = [step_func_dict[step] for step in dance_steps]

In [8]:
prog_list = list(alpha_16)
prog_list_orig = list(alpha_16)
dance_ct = 0
found_cycle = False
while not found_cycle:
    for op, args in x_dance_steps:
        if op == 's':
            spin_len = args[0]
            prog_list = prog_list[-spin_len:] + prog_list[:-spin_len]
        elif op == 'x':
            sp1, sp2 = args
            prog_list[sp1], prog_list[sp2] = prog_list[sp2], prog_list[sp1]
        elif op == 'p':
            pp1, pp2 = args
            sp1 = prog_list.index(pp1)
            sp2 = prog_list.index(pp2)
            prog_list[sp1], prog_list[sp2] = prog_list[sp2], prog_list[sp1]
    dance_ct += 1
    if prog_list == prog_list_orig:
        found_cycle = True

In [9]:
dance_ct

44

In [10]:
1_000_000_000 % 44

32

In [11]:
prog_list = list(alpha_16)
for _ in range(32):
    for op, args in x_dance_steps:
        if op == 's':
            spin_len = args[0]
            prog_list = prog_list[-spin_len:] + prog_list[:-spin_len]
        elif op == 'x':
            sp1, sp2 = args
            prog_list[sp1], prog_list[sp2] = prog_list[sp2], prog_list[sp1]
        elif op == 'p':
            pp1, pp2 = args
            sp1 = prog_list.index(pp1)
            sp2 = prog_list.index(pp2)
            prog_list[sp1], prog_list[sp2] = prog_list[sp2], prog_list[sp1]

In [12]:
''.join(prog_list)

'ahgpjdkcbfmneloi'